In [ ]:
%%capture
!pip install blosc
!pip install wandb

In [ ]:
import tensorflow as tf

In [ ]:
import blosc
import pickle
import math
from tqdm import tqdm
import numpy as np
import sklearn.model_selection
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
my_drive = GoogleDrive(gauth)

In [ ]:
import wandb
from wandb.keras import WandbCallback
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
%cd '/content/gdrive/My Drive/BWSI_AI_DJ_2023/'

/content/gdrive/My Drive/BWSI_AI_DJ_2023


In [ ]:
start = 0
end = 4283

interval = 370
file_cnt = math.ceil((end - start) / interval)

path_template = 'all_ps_dbs_4283_2813/all_ps_dbs_%d_to_%d.dat'

all_ps_dbs = [None for _ in range(file_cnt)]

for i in tqdm(range(file_cnt)):
    start_idx = i * interval + start
    end_idx = min((i + 1) * interval + start, end)
    with open(path_template % (start_idx, end_idx), 'rb') as f:
        all_ps_dbs[i] = pickle.loads(blosc.decompress(f.read()))

all_ps_dbs = np.concatenate(all_ps_dbs, axis=0)
X_data = all_ps_dbs
del all_ps_dbs
print(X_data.shape)

100%|██████████| 12/12 [04:24<00:00, 22.05s/it]


(4283, 256, 2813)


In [ ]:
with open('timestamp_labels/timestamp_labels_3.dat', 'rb') as f:
    y_data = pickle.loads(blosc.decompress(f.read()))
print(y_data.shape)

(4283, 240)


In [ ]:
bad_idx = [
    178,  370,  665,  836, 1681, 2270, 2271, 2272, 2273, 2274, 2276,
    2278, 2284, 2302, 2311, 2323, 2601, 2605, 2733, 3267, 3276, 3365,
    3420, 3544, 3602, 3797, 3850, 4098
]

X_data = np.delete(X_data, bad_idx, axis=0)
y_data = np.delete(y_data, bad_idx, axis=0)

In [ ]:
test_split = 0.2

RANDOM_STATE = 0
X_data, y_data = sklearn.utils.shuffle(X_data, y_data, random_state=RANDOM_STATE)

data_num = X_data.shape[0]
test_idx = int(test_split * data_num)

X_test, X_train = np.split(X_data, [test_idx])
y_test, y_train = np.split(y_data, [test_idx])

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3404, 256, 2813)
(3404, 240)
(851, 256, 2813)
(851, 240)


In [ ]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
print(X_train.shape)
print(X_test.shape)

(3404, 256, 2813, 1)
(851, 256, 2813, 1)


In [ ]:
del X_data
del y_data

In [ ]:
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'lr': {
            'max': 0.1,
            'min': 0.0001
        },
        'dropout': {
            'max': 0.6,
            'min': 0.1
        },
        'l2_reg': {
            'max': 0.1,
            'min': 0.00001
        },
        'dense_layers': {
            'values': [1, 2]
        },
        'conv_layers': {
            'values': [1, 2, 3]
        },
        'epochs': {
            'values': [10, 50, 100, 200, 400]
        },
        'nodes1': {
            'values': [128, 256, 512]
        },
        'nodes2': {
            'values': [128, 256, 512]
        },
        'nodes3': {
            'value': 240
        },
        'conv1_filter': {
            'values': [4, 8, 16, 32, 64, 128]
        },
        'conv1_kernel_size': {
            'values': [3, 4, 5, 8]
        },
        'conv1_stride': {
            'values': [1, 2, 3, 4, 8]
        },
        'conv2_filter': {
            'values': [4, 8, 16, 32, 64, 128]
        },
        'conv2_kernel_size': {
            'values': [3, 4, 5, 8]
        },
        'conv2_stride': {
            'values': [1, 2, 3, 4]
        },
        'conv3_filter': {
            'values': [4, 8, 16, 32, 64, 128]
        },
        'conv3_kernel_size': {
            'values': [3, 4, 5, 8]
        },
        'conv3_stride': {
            'value': 1
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(
  sweep=sweep_config,
  project='bwsi_ai_dj_sweep_4'
)

Create sweep with ID: up94b77x
Sweep URL: https://wandb.ai/team-2412/bwsi_ai_dj_sweep_4/sweeps/up94b77x


In [ ]:
def sweep_train():
    for a_file in my_drive.ListFile({'q': "trashed = true"}).GetList():
        # print the name of the file being deleted.
        title = a_file['title']
        print(f'the file "{title}", is about to get deleted permanently.')
        # delete the file permanently.
        a_file.Delete()


    run = wandb.init()

    lr = wandb.config.lr
    dropout = wandb.config.dropout
    l2_reg = wandb.config.l2_reg
    dense_layers = wandb.config.dense_layers
    conv_layers = wandb.config.conv_layers
    epochs = wandb.config.epochs
    nodes1 = wandb.config.nodes1
    nodes2 = wandb.config.nodes2
    nodes3 = wandb.config.nodes3
    conv1_filter = wandb.config.conv1_filter
    conv1_kernel_size = wandb.config.conv1_kernel_size
    conv1_stride = wandb.config.conv1_stride
    conv2_filter = wandb.config.conv2_filter
    conv2_kernel_size = wandb.config.conv2_kernel_size
    conv2_stride = wandb.config.conv2_stride
    conv3_filter = wandb.config.conv3_filter
    conv3_kernel_size = wandb.config.conv3_kernel_size
    conv3_stride = wandb.config.conv3_stride

    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Input(shape=X_train.shape[1:])) # 256 x 2813

    model.add(tf.keras.layers.Conv2D(conv1_filter, (conv1_kernel_size, conv1_kernel_size), strides=(conv1_stride, conv1_stride), padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2_reg)))
    model.add(tf.keras.layers.MaxPool2D())
    model.add(tf.keras.layers.Dropout(dropout))

    if conv_layers >= 2:
        model.add(tf.keras.layers.Conv2D(conv2_filter, (conv2_kernel_size, conv2_kernel_size), strides=(conv2_stride, conv2_stride), padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2_reg)))
        model.add(tf.keras.layers.MaxPool2D())
        model.add(tf.keras.layers.Dropout(dropout))

    if conv_layers >= 3:
        model.add(tf.keras.layers.Conv2D(conv3_filter, (conv3_kernel_size, conv3_kernel_size), strides=(conv3_stride, conv3_stride), padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2_reg)))
        model.add(tf.keras.layers.MaxPool2D())
        model.add(tf.keras.layers.Dropout(dropout))

    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(nodes1, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2_reg)))
    model.add(tf.keras.layers.Dropout(dropout))

    if dense_layers >= 2:
        model.add(tf.keras.layers.Dense(nodes2, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2_reg)))
        model.add(tf.keras.layers.Dropout(dropout))

    model.add(tf.keras.layers.Dense(nodes3, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.L2(l2_reg)))

    model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss=tf.keras.losses.BinaryCrossentropy(), metrics=[])

    model.summary()

    wandb_callbacks = [
        WandbCallback(monitor='val_loss', mode='min', save_model=False)
    ]

    model.fit(X_train, y_train, batch_size=16, epochs=epochs, validation_split=0.1, shuffle=True, verbose=1, callbacks=wandb_callbacks)

In [ ]:
wandb.agent(sweep_id, function=sweep_train)

wandb: Agent Starting Run: b2sf05ab with config:
wandb: 	conv1_filter: 128
wandb: 	conv1_kernel_size: 3
wandb: 	conv1_stride: 2
wandb: 	conv2_filter: 64
wandb: 	conv2_kernel_size: 8
wandb: 	conv2_stride: 1
wandb: 	conv3_filter: 128
wandb: 	conv3_kernel_size: 8
wandb: 	conv3_stride: 1
wandb: 	conv_layers: 3
wandb: 	dense_layers: 1
wandb: 	dropout: 0.3973802574545688
wandb: 	epochs: 100
wandb: 	l2_reg: 0.07050310796239147
wandb: 	lr: 0.07002005896669324
wandb: 	nodes1: 256
wandb: 	nodes2: 256
wandb: 	nodes3: 240
wandb: Currently logged in as: rohand2412 (team-2412). Use `wandb login --relogin` to force relogin


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 1407, 128)    1280      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 703, 128)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 64, 703, 128)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 703, 64)       524352    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 351, 64)      0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 32, 351, 64)       0

192/192 [==============================] - 83s 170ms/step - loss: 4386.1157 - val_loss: 531.3582
Epoch 2/100
192/192 [==============================] - 17s 89ms/step - loss: 233.0085 - val_loss: 109.4458
Epoch 3/100
192/192 [==============================] - 17s 88ms/step - loss: 77.6575 - val_loss: 55.3278
Epoch 4/100
192/192 [==============================] - 17s 88ms/step - loss: 42.5560 - val_loss: 32.0895
Epoch 5/100
192/192 [==============================] - 17s 88ms/step - loss: 25.2166 - val_loss: 19.7598
Epoch 6/100
192/192 [==============================] - 17s 88ms/step - loss: 15.6691 - val_loss: 12.3668
Epoch 7/100
192/192 [==============================] - 17s 88ms/step - loss: 10.4214 - val_loss: 8.7541
Epoch 8/100
192/192 [==============================] - 17s 88ms/step - loss: 8.2560 - val_loss: 8.6242
Epoch 9/100
192/192 [==============================] - 17s 89ms/step - loss: 8.6577 - val_loss: 9.6344
Epoch 10/100
192/192 [==============================] - 17s 88ms/s

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▁▁▅▁▁▁▃▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,13
best_val_loss,4.78446
epoch,99
loss,21.18734
val_loss,21.09533


wandb: Agent Starting Run: 87rat1dl with config:
wandb: 	conv1_filter: 8
wandb: 	conv1_kernel_size: 8
wandb: 	conv1_stride: 1
wandb: 	conv2_filter: 4
wandb: 	conv2_kernel_size: 4
wandb: 	conv2_stride: 3
wandb: 	conv3_filter: 64
wandb: 	conv3_kernel_size: 4
wandb: 	conv3_stride: 1
wandb: 	conv_layers: 3
wandb: 	dense_layers: 1
wandb: 	dropout: 0.45743598238128313
wandb: 	epochs: 50
wandb: 	l2_reg: 0.05843459011820049
wandb: 	lr: 0.020017598550269432
wandb: 	nodes1: 128
wandb: 	nodes2: 256
wandb: 	nodes3: 240


...

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 704, 128)      3328      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 352, 128)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 32, 352, 128)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 88, 64)         131136    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 44, 64)        0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 4, 44, 64)         0

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▁▆█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
val_loss,▁██▅▆▆▆▆▆▅▆▆▆▅▅▆▆▆▆▆▆▆▅▅▆▅▆▆▅▆▆▆▆▆▅▆▆▅▆▆
best_epoch,5
best_val_loss,0.46729
epoch,399
loss,0.49308
val_loss,0.49149


wandb: Agent Starting Run: tjnfh9cb with config:
wandb: 	conv1_filter: 8
wandb: 	conv1_kernel_size: 4
wandb: 	conv1_stride: 4
wandb: 	conv2_filter: 16
wandb: 	conv2_kernel_size: 4
wandb: 	conv2_stride: 2
wandb: 	conv3_filter: 8
wandb: 	conv3_kernel_size: 3
wandb: 	conv3_stride: 1
wandb: 	conv_layers: 1
wandb: 	dense_layers: 1
wandb: 	dropout: 0.26781629888597236
wandb: 	epochs: 50
wandb: 	l2_reg: 0.02589255763403749
wandb: 	lr: 0.07400610238091657
wandb: 	nodes1: 256
wandb: 	nodes2: 256
wandb: 	nodes3: 240


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 704, 8)        136       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 352, 8)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 32, 352, 8)        0         
                                                                 
 flatten (Flatten)           (None, 90112)             0         
                                                                 
 dense (Dense)               (None, 256)               23068928  
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                        

wandb: Ctrl + C detected. Stopping sweep.
